In [ ]:
import logging
from datetime import date

import pandas as pd
from sqlalchemy import MetaData, Selectable, Table
from sqlalchemy.dialects.mysql import insert
from sqlalchemy.engine import Engine
from sqlalchemy.exc import NoSuchTableError
from sqlalchemy.sql import and_, case, func, or_, select

from utils import mysql_connection

In [ ]:
def nation_sql(metadata: MetaData, year: int) -> Selectable:
    u = metadata.tables["weaselbot.combined_users"]
    a = metadata.tables["weaselbot.combined_aos"]
    b = metadata.tables["weaselbot.combined_beatdowns"]
    bd = metadata.tables["weaselbot.combined_attendance"]
    r = metadata.tables["weaselbot.combined_regions"]
    cu = metadata.tables["weaselbot.combined_users_dup"]

    sql = select(
        u.c.email,
        u.c.user_name,
        cu.c.slack_user_id,
        a.c.ao_id,
        a.c.ao_name.label("ao"),
        b.c.bd_date.label("date"),
        case((or_(bd.c.user_id == b.c.q_user_id, bd.c.user_id == b.c.coq_user_id), 1), else_=0).label("q_flag"),
        b.c.backblast,
        r.c.schema_name.label("home_region"),
    )
    sql = sql.select_from(
        bd.join(u, bd.c.user_id == u.c.user_id)
        .join(cu, and_(u.c.email == cu.c.email, u.c.home_region_id == cu.c.region_id))
        .join(b, bd.c.beatdown_id == b.c.beatdown_id)
        .join(a, b.c.ao_id == a.c.ao_id)
        .join(r, u.c.home_region_id == r.c.region_id)
    )
    sql = sql.where(
        b.c.bd_date.between(f"{year}-01-01", func.curdate()), u.c.email != "none", u.c.user_name != "PAXminer"
    )

    return sql


def region_sql(metadata: MetaData):
    """Pick out only the regions that want their weaselshaker awards."""
    t = metadata.tables["weaselbot.regions"]
    return select(t).where(t.c.send_achievements == 1)


def award_view(row, engine: Engine, metadata: MetaData, year: int) -> Selectable:
    aa = Table("achievements_awarded", metadata, autoload_with=engine, schema=row.paxminer_schema)
    al = Table("achievements_list", metadata, autoload_with=engine, schema=row.paxminer_schema)

    sql = (
        select(aa, al.c.code)
        .select_from(aa.join(al, aa.c.achievement_id == al.c.id))
        .where(func.year(aa.c.date_awarded) == year)
    )
    return sql


def award_list(row, engine: Engine, metadata: MetaData) -> Selectable:
    try:
        al = metadata.tables[f"{row.paxminer_schema}.achievements_list"]
    except KeyError:
        al = Table("achievements_list", metadata, autoload_with=engine, schema=row.paxminer_schema)

    return select(al)

In [ ]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
metadata.reflect(engine, schema="weaselbot")

df_regions = pd.read_sql(region_sql(metadata), engine)
nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

In [ ]:
engine.dispose()

In [ ]:
nation_df.assign(week=nation_df.date.dt.isocalendar().week).loc[lambda x: (x.home_region == "f3austin") & (x.slack_user_id == "U06064CJR5F")].sort_values("week", ascending=False)

In [ ]:
# note: 4/6 beatdown isn't showing up in the Nation beatdown data table